In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
import helper_scgen as hscg
print(sys.executable)
import anndata as ad
import time
import scgen
import scvi
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')
h.print_main_versions()

In [ ]:
# proof principle with models from MH29_v2_c

In [ ]:
pre = "MH102"
drive = 'F'
_,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

#base_model_path = os.path.join(drive + ":\\","monkey_IZI","analysis","cluster_runs",cluster_pre,"output")

celltypes = ['CD4 T', 'CD8 T','B','Mono','NK','other T','other']

all_species = ['cyno','human']
all_conditions = ['00hr','06hr','24hr']
n_latent = 10
shifts_within_species = ['00hr->06hr','00hr->24hr','06hr->24hr']

In [ ]:
#base_model_path_MH31_c = 
#_,_,_,base_anndata_objects_MH27 = h.return_local_paths(drive = drive,pre = 'MH27',add_path = False)

### Humanization proof-of-principle
- train set: human1, cyno1
- test set: human2, cyno2

In [ ]:
cluster_pre = "MH31_c"
base_model_path = os.path.join(drive + ":\\","monkey_IZI","analysis","cluster_runs",cluster_pre,"output")

In [ ]:
#for celltype in ['CD4 T']:
celltype = 'CD4 T'
loader = os.path.join(base_model_path,cluster_pre + '_scgen_model_' + celltype +  '.pt')
model_loaded = scgen.SCGEN.load(loader)

adata_model = model_loaded.adata
latent_adata_train = hscg.get_latent_representation_object(model=model_loaded,adata=adata_model)

In [ ]:
#train set
latent_adata_train_human_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train,
                                                               col_names=['timepoint','species'],
                                                               val_names=['00hr','human'])


latent_adata_train_cyno_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train,
                                                               col_names=['timepoint','species'],
                                                               val_names=['00hr','cyno'])

#get species shift
delta_cross_species_train = hscg.get_delta_in_latent_space(adata_lat_base=latent_adata_train_cyno_00hr,
                              adata_lat_dest=latent_adata_train_human_00hr)

In [ ]:
# train set

In [ ]:
set_type = 'train'
adata_train = sc.read_h5ad(os.path.join(base_model_path,'MH31_c_adata_train.h5ad'))

adata_train_00hr = hscg.filter_multiple_adata_obs(adata=adata_train,
                                                 col_names=["predicted.celltype.l1",'timepoint'],
                                                 val_names=['CD4 T','00hr'])

In [ ]:
adata_train_00hr_cyno = hscg.filter_multiple_adata_obs(adata=adata_train,col_names=["predicted.celltype.l1",'timepoint','species'],
                                                      val_names=['CD4 T','00hr','cyno'])
adata_train_00hr_human = hscg.filter_multiple_adata_obs(adata=adata_train,col_names=["predicted.celltype.l1",'timepoint','species'],
                                                      val_names=['CD4 T','00hr','human'])

In [ ]:
h.prepare_umap(adata_train_00hr)
palette = ['#178f92',"#d8c29d"]
h.umap_plot_v2(adata=adata_train_00hr,
             color='species',
             base_plots_path = base_plots_path,
             plot_name = pre +'_'+ set_type +'_umap_GE_pre_' + str(celltype) + '.pdf',
            palette = palette) 

In [ ]:
latent_adata_train_00hr = hscg.get_latent_representation_object(model=model_loaded,adata=adata_train_00hr)

In [ ]:
latent_adata_train_00hr_human_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train_00hr ,
                                                               col_names=['timepoint','species'],
                                                               val_names=['00hr','human'])
latent_adata_train_00hr_cyno_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train_00hr ,
                                                               col_names=['timepoint','species'],
                                                               val_names=['00hr','cyno'])

latent_adata_train_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_train_00hr_cyno_00hr,
                                                                   delta=delta_cross_species_train,
                                                                   cols_suffix=['timepoint','species'],
                                                                   suffix=['_shifted','_shifted'])

adata_latent_comp_train = ad.concat([latent_adata_train_00hr_human_00hr,latent_adata_train_00hr_cyno_00hr,latent_adata_train_00hr_cyno_shifted])

h.prepare_umap(adata_latent_comp_train)

palette = ['#178f92','#175f5d',"#d8c29d"]
h.umap_plot_v2(adata=adata_latent_comp_train,
             color='species',
             base_plots_path = base_plots_path,
             plot_name = pre +'_'+ set_type +'_umap_latent' + str(celltype) + '.pdf',
            palette = palette) 

In [ ]:
#encode - shift - decode

In [ ]:
latent_adata_train_00hr_cyno = hscg.get_latent_representation_object(model=model_loaded,adata= adata_train_00hr_cyno)

latent_adata_train_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_train_00hr_cyno,delta=delta_cross_species_train)

decoded_adata_train_00hr_cyno_shifted =hscg.decode(model=model_loaded,
                                          latent_adata=latent_adata_train_00hr_cyno_shifted,
                                          input_adata=adata_train_00hr_cyno) 

decoded_adata_train_00hr_cyno_shifted.obs['timepoint'] = decoded_adata_train_00hr_cyno_shifted.obs['timepoint'].astype(str) + '_shifted'
decoded_adata_train_00hr_cyno_shifted.obs['species'] = decoded_adata_train_00hr_cyno_shifted.obs['species'].astype(str) + '_shifted'

adata_train_00hr_decoded = ad.concat([adata_train_00hr_cyno,adata_train_00hr_human,decoded_adata_train_00hr_cyno_shifted])

h.prepare_umap(adata_train_00hr_decoded)

palette = ['#178f92','#175f5d',"#d8c29d"]
h.umap_plot_v2(adata=adata_train_00hr_decoded,
             color='species',
             base_plots_path = base_plots_path,
             plot_name = pre +'_'+ set_type +'_umap_GE_' + str(celltype) + '.pdf',
            palette = palette) 

In [ ]:
#test set

In [ ]:
set_type = 'test'
adata_test = sc.read_h5ad(os.path.join(base_model_path,'MH31_c_adata_test.h5ad'))

adata_test_00hr = hscg.filter_multiple_adata_obs(adata=adata_test,
                                                 col_names=["predicted.celltype.l1",'timepoint'],
                                                 val_names=['CD4 T','00hr'])

In [ ]:
adata_test_00hr_cyno = hscg.filter_multiple_adata_obs(adata=adata_test,col_names=["predicted.celltype.l1",'timepoint','species'],
                                                      val_names=['CD4 T','00hr','cyno'])
adata_test_00hr_human = hscg.filter_multiple_adata_obs(adata=adata_test,col_names=["predicted.celltype.l1",'timepoint','species'],
                                                      val_names=['CD4 T','00hr','human'])

In [ ]:
h.prepare_umap(adata_test_00hr)
palette = ['#178f92',"#d8c29d"]
h.umap_plot_v2(adata=adata_test_00hr,
             color='species',
             base_plots_path = base_plots_path,
             plot_name = pre +'_'+ set_type +'_umap_GE_pre_' + str(celltype) + '.pdf',
            palette = palette) 

In [ ]:
latent_adata_test_00hr = hscg.get_latent_representation_object(model=model_loaded,adata=adata_test_00hr)

In [ ]:
latent_adata_test_00hr_human_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_test_00hr ,
                                                               col_names=['timepoint','species'],
                                                               val_names=['00hr','human'])
latent_adata_test_00hr_cyno_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_test_00hr ,
                                                               col_names=['timepoint','species'],
                                                               val_names=['00hr','cyno'])

latent_adata_test_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_test_00hr_cyno_00hr,
                                                                   delta=delta_cross_species_train,
                                                                   cols_suffix=['timepoint','species'],
                                                                   suffix=['_shifted','_shifted'])

adata_latent_comp_test = ad.concat([latent_adata_test_00hr_human_00hr,latent_adata_test_00hr_cyno_00hr,latent_adata_test_00hr_cyno_shifted])

h.prepare_umap(adata_latent_comp_test)

palette = ['#178f92','#175f5d',"#d8c29d"]
h.umap_plot_v2(adata=adata_latent_comp_test,
             color='species',
             base_plots_path = base_plots_path,
             plot_name = pre +'_'+ set_type +'_umap_latent' + str(celltype) + '.pdf',
            palette = palette) 
#sc.pl.umap(adata_latent_comp,color = ['timepoint','species','individual'])

In [ ]:
#encode - shift - decode

In [ ]:
latent_adata_test_00hr_cyno = hscg.get_latent_representation_object(model=model_loaded,adata= adata_test_00hr_cyno)

latent_adata_test_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_test_00hr_cyno,delta=delta_cross_species_train)

decoded_adata_test_00hr_cyno_shifted =hscg.decode(model=model_loaded,
                                          latent_adata=latent_adata_test_00hr_cyno_shifted,
                                          input_adata=adata_test_00hr_cyno) 

decoded_adata_test_00hr_cyno_shifted.obs['timepoint'] = decoded_adata_test_00hr_cyno_shifted.obs['timepoint'].astype(str) + '_shifted'
decoded_adata_test_00hr_cyno_shifted.obs['species'] = decoded_adata_test_00hr_cyno_shifted.obs['species'].astype(str) + '_shifted'

adata_test_00hr_decoded = ad.concat([adata_test_00hr_cyno,adata_test_00hr_human,decoded_adata_test_00hr_cyno_shifted])

h.prepare_umap(adata_test_00hr_decoded)

palette = ['#178f92','#175f5d',"#d8c29d"]
h.umap_plot_v2(adata=adata_test_00hr_decoded,
             color='species',
             base_plots_path = base_plots_path,
             plot_name = pre +'_'+ set_type +'_umap_GE_' + str(celltype) + '.pdf',
            palette = palette) 

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()